%% [markdown]<br>
# Exercise 1: Machine Learning Interatomic Potentials<br>
<br>
In this tutorial, we will:<br>
<br>
1. Optimize bulk Pd using the MACE MP0 model<br>
2. Build a Pd(111) surface slab from the optimized bulk lattice.<br>
3. Run a short molecular dynamics (MD) simulation on the slab.<br>
4. Add a water molecule as an adsorbate and relax the system.<br>
5. Calculate the adsorption energy:<br>
<br>
\[<br>
E_\text{ads} = E_\text{slab+H₂O} - (E_\text{slab} + E_\text{H₂O})<br>
\]<br>
<br>
A negative value indicates favorable adsorption (exothermic).

%%=

In [ ]:
from ase.build import bulk, make_supercell, fcc111, molecule, add_adsorbate
from mace.calculators import mace_mp
from ase.constraints import StrainFilter
from ase.optimize import BFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.visualize import view
from ase.io import read
from ase.md.langevin import Langevin
from ase import units

%% [markdown]<br>
## Step 1: Optimize bulk Pd

%%<br>
Build bulk Pd (fcc)

In [ ]:
element = 'Pd'
atoms = bulk(element, 'fcc', a=3.859)

Assign MACE calculator

In [ ]:
macemp = mace_mp(dispersion=True, float=64)
atoms.calc = macemp

Optimize cell with strain filter

In [ ]:
sf = StrainFilter(atoms)
opt_bulk = BFGS(sf, trajectory='Pd_bulk_opt.traj', logfile='Pd_bulk_opt.log')
opt_bulk.run(fmax=0.01)

Optimized lattice constant

In [ ]:
optimized_a = atoms.get_cell_lengths_and_angles()[0]
print(f"Optimized lattice constant a = {optimized_a:.3f} Å")

In [ ]:
view(atoms, viewer='ngl')

%% [markdown]<br>
## Step 2: Build Pd(111) slab<br>
<br>
We'll construct a 5x5x3 Pd(111) surface slab from the optimized bulk lattice.

%%

In [ ]:
slab = fcc111('Pd', size=(5,5,3), a=optimized_a)
slab.calc = macemp

In [ ]:
E_slab = slab.get_potential_energy()
print(f"Clean Pd(111) slab energy: {E_slab:.3f} eV")

In [ ]:
view(slab, viewer='ngl')

%% [markdown]<br>
## Step 3: Optimize isolated H₂O molecule

%%

In [ ]:
h2o = molecule('H2O')
h2o.calc = macemp

In [ ]:
opt_h2o = BFGS(h2o, trajectory='H2O_opt.traj', logfile='H2O_opt.log')
opt_h2o.run(fmax=0.01)

In [ ]:
E_h2o = h2o.get_potential_energy()
print(f"Isolated H₂O energy: {E_h2o:.3f} eV")

In [ ]:
view(h2o, viewer='ngl')

%% [markdown]<br>
## Step 4: Add H₂O to Pd(111) surface and relax

%%<br>
Center water molecule above slab

In [ ]:
x_center = slab.get_cell()[0,0] / 2
y_center = slab.get_cell()[1,1] / 2

In [ ]:
add_adsorbate(slab, h2o, 1.5, position=(x_center, y_center))
slab.center(vacuum=10.0, axis=2)
slab.calc = macemp

Optimize adsorbed system

In [ ]:
opt_ads = BFGS(slab, trajectory='Pd_H2O_ads.traj', logfile='Pd_H2O_ads.log')
opt_ads.run(fmax=0.01)

In [ ]:
E_Pd_H2O = slab.get_potential_energy()
print(f"Pd(111) + H₂O total energy: {E_Pd_H2O:.3f} eV")

In [ ]:
view(slab, viewer='ngl')

%% [markdown]<br>
## Step 5: Calculate adsorption energy

%%

In [ ]:
E_adsorption = E_Pd_H2O - (E_slab + E_h2o)
print(f"Adsorption energy of H₂O on Pd(111): {E_adsorption:.3f} eV")

%% [markdown]<br>
## Step 6: Summary<br>
<br>
| Quantity           | Symbol       | Energy (eV) |<br>
|-------------------|-------------|-------------|<br>
| Clean slab        | E_slab      | `{E_slab:.3f}` |<br>
| Isolated H₂O      | E_h2o       | `{E_h2o:.3f}` |<br>
| Adsorbed system   | E_Pd_H2O    | `{E_Pd_H2O:.3f}` |<br>
| **Adsorption energy** | **E_adsorption** | **`{E_adsorption:.3f}`** |<br>
<br>
Negative adsorption energy → exothermic adsorption (favorable binding).

%% [markdown]<br>
## Step 7: Run MD simulation of H₂O on Pd(111)<br>
<br>
We'll perform a short molecular dynamics (MD) simulation at 300 K using a Langevin thermostat.

%%

In [ ]:
ads_slab = slab.copy()
ads_slab.calc = macemp

Initialize velocities

In [ ]:
MaxwellBoltzmannDistribution(ads_slab, temperature_K=300)

Langevin MD

In [ ]:
dyn = Langevin(
    ads_slab,
    timestep=1 * units.fs,
    temperature_K=300,
    friction=0.02,
    logfile='Pd_H2O_md.log',
    trajectory='Pd_H2O_md.traj'
)

In [ ]:
dyn.run(1000)
print("MD simulation complete!")

%% [markdown]<br>
## Step 8: Visualize MD trajectory

%%

In [ ]:
md_frames = read('Pd_H2O_md.traj', index=':')
view(md_frames, viewer='ngl')